#### **LLM Gemini**

In [ ]:
from langchain_google_genai import GoogleGenerativeAI as GoogleAI

llm = GoogleAI(model="gemini-2.0-flash")

#### **Chat Gemini**

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI as ChatGoogleAI

chat = ChatGoogleAI(model="gemini-2.0-flash")

#### **Especialista em Esportes - Cachê em Memória** 

In [ ]:

from langchain_core.messages import HumanMessage, SystemMessage

mensagens = [
    SystemMessage(content='Você é um especialista em esportes no geral.'),
    HumanMessage(content=input('Digite sua dúvida: '))
]

from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

chat.invoke(mensagens)

#### **Perguntas de Acompanhamento - Prompt Templates**

In [ ]:

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from Templates import exemplos

example_prompt = ChatPromptTemplate.from_messages(
    [('human','{pergunta}'),
    ('ai','{resposta}')]
)

example_prompt.format_messages(**exemplos[0])

few_shot_templates = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_template = ChatPromptTemplate.from_messages(
    [few_shot_templates,
    ('human','{input}')]
)

prompt = prompt_template.invoke(input=input('Digite sua dúvida: '))

print(chat.invoke(prompt))

#### **Review de Clientes - Output Parsers**

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

class ReviewClientes(BaseModel):
    """Review de clientes sobre um produto"""
    produto: str = Field(description='Retire uma breve descrição do produto')
    satisfacao_entrega: bool = Field(description='True se o cliente ficou satisfeito com a entrega, False se não.')
    produto_avaliacao: list[str] = Field(description='Retire trecho(os) sobre a satisfação do cliente com o produto. Retorne em uma lista')
    atendimento: str = Field(description='Extraia da review se o cliente ficou satisfeito com o atendimento.')
    satisfacao: Optional[int] = Field(description='Avalie de 1 a 10 a satisfação geral do cliente com o produto.')
    
llm_estruturada = chat.with_structured_output(ReviewClientes)
resposta = llm_estruturada.invoke(input('Escreva seu review: '))

print(resposta)

#### **Tradução e Resumo de Textos - Chains com LCEL**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt1 = ChatPromptTemplate.from_template('Traduza este texto para o Português Brasileiro: {texto}')

chain1 = prompt1 | chat | StrOutputParser()

prompt2 = ChatPromptTemplate.from_template('Resuma o texto: `{texto}')

chain2 = prompt2 | chat | StrOutputParser()

chain3 = chain1 | chain2

print(chain3.invoke(input('Digite o seu texto: ')))

#### **Criador de ideias para Agentes de IA - Runnables**

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import ChatPromptTemplate

prompt1 = ChatPromptTemplate.from_template('Crie um nome para o seguinte Agente IA: {agente}')
chain_nome = prompt1 | chat | StrOutputParser()

prompt2 = ChatPromptTemplate.from_template('Crie um Brainstorming de como o Agente IA a seguir irá funcionar e solucionar problemas: {agente}')
chain_brainstorming = prompt2 | chat | StrOutputParser()

prompt3 = ChatPromptTemplate.from_template("""Dado o Agente de IA com o seguinte nome e as seguintes
ideias, desenvolva o passo a passo para criar esse Agente de IA, usando Python e LangChain.

Nome do Agente: {nome_agente}
Brainstorming: {brainstorming}""")

parallel = RunnableParallel({'nome_agente': chain_nome, 'brainstorming': chain_brainstorming})
chain = parallel | prompt3 | chat | StrOutputParser()

print(chain.invoke({'agente', input('Digite o seu Agente: ')}))

#### **IA Universitária de CDIA - Route + Memory**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages([
    ('system', '''Você é um professor de programação capaz de dar respostas muito 
     detalhadas e didáticas sobre Python e suas bibliotecas. Responda as perguntas de um aluno.'''),
    ('placeholder', '{memoria}'),
    ('human', '{pergunta}'),
    ])
chain_python = prompt | chat

prompt = ChatPromptTemplate.from_messages([
    ('system', '''Você é um professor de programação capaz de dar respostas muito 
     detalhadas e didáticas sobre C/C++ e suas bibliotecas. Responda as perguntas de um aluno.'''),
    ('placeholder', '{memoria}'),
    ('human', '{pergunta}'),
    ])
chain_c = prompt | chat

prompt = ChatPromptTemplate.from_messages([
    ('system', '''Você é um professor de programação capaz de dar respostas muito 
     detalhadas e didáticas sobre Java e suas bibliotecas. Responda as perguntas de um aluno.'''),
    ('placeholder', '{memoria}'),
    ('human', '{pergunta}'),
    ])
chain_java = prompt | chat

prompt = ChatPromptTemplate.from_messages([
    ('system', '''Você é um professor de matemática do ensino superior capaz de dar 
     respostas muito detalhadas e didáticas. Responda as perguntas de um aluno.'''),
    ('placeholder', '{memoria}'),
    ('human', '{pergunta}'),
    ])
chain_matematica = prompt | chat

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Responda a perguntas genéricas.'),
    ('placeholder', '{memoria}'),
    ('human','{pergunta}')
    ])
chain_generica = prompt | chat

prompt = ChatPromptTemplate.from_template('Você deve categorizar a seguinte pergunta : {pergunta}')

class Categorizador(BaseModel):
    """Categoriza as perguntas de alunos de programação do ensino superior"""
    area_conhecimento: str = Field(description='A área de conhecimento da pergunta feita pelo aluno. \
        Deve ser "python", "C/C++", "java" ou "matematica". Caso não se encaixe em nenhuma delas, retorne "outra"')
    
model_estruturado = prompt | chat.with_structured_output(Categorizador)

def route(input):
    if input['categoria'].area_conhecimento == 'python':
        return chain_python
    if input['categoria'].area_conhecimento == 'C/C++':
        return chain_c
    if input['categoria'].area_conhecimento == 'java':
        return chain_java
    if input['categoria'].area_conhecimento == 'matematica':
        return chain_matematica
    return chain_generica

chain = RunnablePassthrough().assign(categoria=model_estruturado) | route | StrOutputParser()

store = {}
def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain_com_memória = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key='pergunta',
    history_messages_key='memoria'
)

In [ ]:
config = {'configurable': {'session_id':'João'}}
resposta = chain_com_memória.invoke({'pergunta': input('Olá, aluno! Digite sua dúvida: ')}, config=config)
print(resposta)

### **Transformando Arquivo NBA_Players.csv em Vetores**

#### **Carregador de CSVs - Document Loader**

In [11]:
from langchain_community.document_loaders.csv_loader import CSVLoader

caminho = "C:/Users/joao_/OneDrive/Área de Trabalho/2° Período/Int. à Ciência de Dados/CSVs/nbaplayers.csv"
loader = CSVLoader(caminho)
docs = loader.load()

#### **Splitting de Dados - Text Splitting**

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter as TxtSpliter

char_split = TxtSpliter(
    chunk_size=400,
    chunk_overlap=40,
    separators=['id','\n\n','\n',',',' ', '']
)

texto = [doc.page_content for doc in docs]

documents = (char_split.create_documents(texto))
documents

[Document(metadata={}, page_content='id: 0\nplayer_name: Dennis Rodman\nteam_abbreviation: CHI\nage: 36.0\nplayer_height: 198.12\nplayer_weight: 99.79024\ncollege: Southeastern Oklahoma State\ncountry: USA\ndraft_year: 1986\ndraft_round: 2\ndraft_number: 27\ngp: 55\npts: 5.7\nreb: 16.1\nast: 3.1\nnet_rating: 16.1\noreb_pct: 0.18600000000000003\ndreb_pct: 0.32299999999999995\nusg_pct: 0.1\nts_pct: 0.479\nast_pct: 0.113\nseason: 1996-97'),
 Document(metadata={}, page_content='id: 1\nplayer_name: Dwayne Schintzius\nteam_abbreviation: LAC\nage: 28.0\nplayer_height: 215.9\nplayer_weight: 117.93392\ncollege: Florida\ncountry: USA\ndraft_year: 1990\ndraft_round: 1\ndraft_number: 24\ngp: 15\npts: 2.3\nreb: 1.5\nast: 0.3\nnet_rating: 12.3\noreb_pct: 0.078\ndreb_pct: 0.151\nusg_pct: 0.175\nts_pct: 0.43\nast_pct: 0.048\nseason: 1996-97'),
 Document(metadata={}, page_content='id: 2\nplayer_name: Earl Cureton\nteam_abbreviation: TOR\nage: 39.0\nplayer_height: 205.74\nplayer_weight: 95.25432\ncolleg

#### **Transformando Dados Splittados em Vetores - Embeddings**

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings as GoogleEmbeddings
from langchain_community.vectorstores.chroma import Chroma

embedding_model = GoogleEmbeddings(model="models/embedding-001")

#### **Salvando os Dados em Discos no Chroma - Vector Store**

In [22]:
diretorio = 'C:/Users/joao_/OneDrive/Área de Trabalho/Aplicações de IA com LangChain - Asimov Academy/projetos/Chroma_Testes/chroma_Testes_bd'

batch_size = 5000
vectordb = None

for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    if vectordb is None:
        vectordb = Chroma.from_documents(
            documents=batch,
            embedding=embedding_model,
            persist_directory=diretorio
        )
    else:
        vectordb.add_documents(batch)

#### **Fazendo uma Busca Semântica através de uma Pergunta - MMR - Retrieval**

In [ ]:
pergunta = 'Quem é Stephen Curry?'

docs = vectordb.max_marginal_relevance_search(pergunta, k=3, fetch_k=5)
len(docs)

In [20]:
for doc in docs:
    print(doc.page_content)
    print(f'====== {doc.metadata}\n\n')

dreb_pct: 0.281
usg_pct: 0.297
ts_pct: 0.532
ast_pct: 0.122
season: 2013-14
id: 7708
player_name: Stephen Curry
team_abbreviation: GSW
age: 26.0
player_height: 190.5
player_weight: 83.91452
college: Davidson
country: USA
draft_year: 2009
draft_round: 1
draft_number: 7
gp: 78
pts: 24.0
reb: 4.3
ast: 8.5
net_rating: 9.4
oreb_pct: 0.018000000000000002
dreb_pct: 0.10800000000000001
usg_pct: 0.281
ts_pct: 0.61
ast_pct: 0.37200000000000005
====== {}


draft_round: 1
draft_number: 8
gp: 66
pts: 6.9
reb: 3.3
ast: 1.3
net_rating: -7.9
oreb_pct: 0.025
dreb_pct: 0.142
usg_pct: 0.18600000000000003
ts_pct: 0.485
ast_pct: 0.11800000000000001
season: 2018-19
id: 10382
player_name: Stephen Curry
team_abbreviation: GSW
age: 31.0
player_height: 190.5
player_weight: 86.18248
college: Davidson
country: USA
draft_year: 2009
draft_round: 1
draft_number: 7
gp: 69
pts: 27.3
reb: 5.3
ast: 5.2
====== {}


dreb_pct: 0.158
usg_pct: 0.139
ts_pct: 0.0
ast_pct: 0.1
season: 2019-20
id: 10911
player_name: Stephen Curr